In [1]:
%env CUDA_VISIBLE_DEVICES=0 TOKENIZERS_PARALLELISM=true

env: CUDA_VISIBLE_DEVICES=0 TOKENIZERS_PARALLELISM=true


### Task 1

In [2]:
from task1.train import train

In [3]:
train(with_amp=False, scaler_type=None)

Loss: 0.6165 Accuracy: 92.918: 100%|██████████| 40/40 [00:39<00:00,  1.02it/s] 


In [4]:
train(with_amp=True, scaler_type="static")

Loss: 0.5857 Accuracy: 98.6872: 100%|██████████| 40/40 [00:38<00:00,  1.05it/s]


In [5]:
train(with_amp=True, scaler_type="dynamic")

Loss: 0.5847 Accuracy: 98.7843: 100%|██████████| 40/40 [00:38<00:00,  1.05it/s]


### Task 2

In [2]:
from task2.run_epoch import run_epoch, DataMode
from task2.dataset import get_texts

In [4]:
data_path = "task2/wikitext-103-raw-v1/train-00000-of-00002.txt"
texts = list(get_texts(data_path, max_lines=300000))

In [4]:
run_epoch(DataMode.BRAIN, texts)

100%|██████████| 18751/18751 [31:05<00:00, 10.05it/s]


,pad_token_ratio,time
count,18746.000000,18746.000000
mean,0.844768,0.099114
std,0.036115,0.001630
min,0.679980,0.032973
25%,0.821191,0.098730
50%,0.846289,0.098950
75%,0.869824,0.099207
max,0.966797,0.126127


In [5]:
run_epoch(DataMode.BIG_BRAIN, texts)

100%|██████████| 18751/18751 [14:33<00:00, 21.47it/s]


,pad_token_ratio,time
count,18746.000000,18746.000000
mean,0.650151,0.046283
std,0.076858,0.012042
min,0.000000,0.015003
25%,0.599435,0.037296
50%,0.653017,0.043890
75%,0.704913,0.052716
max,0.880918,0.110691


In [6]:
run_epoch(DataMode.ULTRA_BIG_BRAIN, texts)

100%|██████████| 18751/18751 [14:37<00:00, 21.36it/s]


,pad_token_ratio,time
count,18746.000000,18746.000000
mean,0.650458,0.046509
std,0.077305,0.012095
min,0.000000,0.012021
25%,0.599560,0.037486
50%,0.653737,0.044064
75%,0.704827,0.052992
max,0.895508,0.108156


In [7]:
run_epoch(DataMode.ULTRA_BIG_BRAIN, texts, k=5)

100%|██████████| 18817/18817 [06:34<00:00, 47.74it/s]


,pad_token_ratio,time
count,18812.000000,18812.000000
mean,0.068673,0.020735
std,0.071873,0.013861
min,0.000000,0.006112
25%,0.012083,0.007663
50%,0.022727,0.018079
75%,0.143750,0.028911
max,0.262500,0.099670


In [5]:
run_epoch(DataMode.ULTRA_DUPER_BIG_BRAIN, texts)

100%|██████████| 300001/300001 [01:05<00:00, 4596.88it/s]


,pad_token_ratio,time
count,5714.0,5714.000000
mean,0.0,0.160782
std,0.0,0.004858
min,0.0,0.084369
25%,0.0,0.158971
50%,0.0,0.161484
75%,0.0,0.163772
max,0.0,0.219913


### Task 3

In [1]:
from task3.run_epoch import profile_vit

In [ ]:
own_profiler_summary = profile_vit("own")

Dataset already extracted
Train Data: 4322
Val Data: 1081


Train:  25%|██▌       | 1/4 [00:28<01:25, 28.57s/it]

In [ ]:
own_profiler_summary

In [ ]:
torch_profiler_summary = profile_vit("torch")
print(torch_profiler_summary)